# Valuing European-style Swaptions with QL Example

We value a European swaption using Black's model and try to replicate a QL example at http://khandrikacm.blogspot.com/2014/03/european-style-interest-rate-swaption.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.rates import *
from financepy.finutils import *
from financepy.market.curves import *
from financepy.models.TuringModelBlack import TuringModelBlack

####################################################################
# FINANCEPY BETA Version 0.185 - This build:  24 Oct 2020 at 21:27 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Building a Ibor Discount Curve

In [3]:
valuationDate = TuringDate(28, 2, 2014)
settlementDate = TuringDate(4, 3, 2014)

We have a vector of dates and zero rates

In [4]:
depoDCCType = TuringDayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = TuringIborDeposit(settlementDate, "1W", 0.0023, depoDCCType); depos.append(depo)
depo = TuringIborDeposit(settlementDate, "1M", 0.0023, depoDCCType); depos.append(depo)
depo = TuringIborDeposit(settlementDate, "3M", 0.0023, depoDCCType); depos.append(depo)
depo = TuringIborDeposit(settlementDate, "6M", 0.0023, depoDCCType); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
swapType = TuringSwapTypes.PAY
fixedDCCType = TuringDayCountTypes.ACT_365F
fixedFreqType = TuringFrequencyTypes.SEMI_ANNUAL

swap = TuringIborSwap(settlementDate, "3Y", swapType, 0.00790, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "4Y", swapType, 0.01200, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "5Y", swapType, 0.01570, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "6Y", swapType, 0.01865, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "7Y", swapType, 0.02160, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "8Y", swapType, 0.02350, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "9Y", swapType, 0.02540, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "10Y", swapType, 0.0273, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "15Y", swapType, 0.0297, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "20Y", swapType, 0.0316, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "25Y", swapType, 0.0335, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "30Y", swapType, 0.0354, fixedFreqType, fixedDCCType); swaps.append(swap)

liborCurve = TuringIborSingleCurve(valuationDate, depos, [], swaps)

Inserting synthetic deposit


In [5]:
exerciseDate = settlementDate.addTenor("5Y")
swapMaturityDate = exerciseDate.addTenor("5Y")

swapFixedCoupon = 0.040852
swapFixedFrequencyType = TuringFrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = TuringDayCountTypes.THIRTY_E_360_ISDA

swapFloatFrequencyType = TuringFrequencyTypes.QUARTERLY
swapFloatDayCountType = TuringDayCountTypes.ACT_360

swapNotional = ONE_MILLION
swapType = TuringSwapTypes.PAY 
calendarType = TuringCalendarTypes.TARGET
busDayAdjustType = TuringBusDayAdjustTypes.MODIFIED_FOLLOWING
dateGenRuleType = TuringDateGenRuleTypes.BACKWARD

In [6]:
swaption = TuringIborSwaption(settlementDate,
                            exerciseDate,
                            swapMaturityDate,
                            swapType,
                            swapFixedCoupon,
                            swapFixedFrequencyType,
                            swapFixedDayCountType, 
                            swapNotional, 
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            calendarType, 
                            busDayAdjustType,
                            dateGenRuleType)

In [7]:
print(swaption)

OBJECT TYPE: TuringIborSwaption
SETTLEMENT DATE: TUE 04 MAR 2014
EXERCISE DATE: MON 04 MAR 2019
SWAP TYPE: TuringSwapTypes.PAY
SWAP MATURITY DATE: MON 04 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: TuringFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: TuringDayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: TuringFrequencyTypes.QUARTERLY
FLOAT DAY COUNT: TuringDayCountTypes.ACT_360



In [8]:
model = TuringModelBlack(0.1533)

In [9]:
swaption.value(settlementDate, liborCurve, model)

23163.13292186464

This is close to a value of 23162 using QL

## Internals

In [10]:
print(swaption)

OBJECT TYPE: TuringIborSwaption
SETTLEMENT DATE: TUE 04 MAR 2014
EXERCISE DATE: MON 04 MAR 2019
SWAP TYPE: TuringSwapTypes.PAY
SWAP MATURITY DATE: MON 04 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: TuringFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: TuringDayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: TuringFrequencyTypes.QUARTERLY
FLOAT DAY COUNT: TuringDayCountTypes.ACT_360
PV01: 4.16168532945034
FWD SWAP RATE: 4.086035922736006
FWD DF TO EXPIRY: 0.9237697293451264


We can see that the forward swap rate almost equals the fixed coupon. The underlying swap is close to being ATM forward.

In [11]:
swaption.printSwapFixedLeg()

START DATE: MON 04 MAR 2019
MATURITY DATE: MON 04 MAR 2024
COUPON (%): 4.0852
FIXED LEG FREQUENCY: TuringFrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: TuringDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE TUE 04 MAR 2014
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
TUE 04 MAR 2014          -            -   1.00000000            -            -
WED 04 SEP 2019  0.5000000     20426.00   0.90872318     18561.58     18561.58
WED 04 MAR 2020  0.5000000     20426.00   0.89262979     18232.86     36794.44
FRI 04 SEP 2020  0.5000000     20426.00   0.87518398     17876.51     54670.94
THU 04 MAR 2021  0.5000000     20426.00   0.85693776     17503.81     72174.75
MON 06 SEP 2021  0.5055556     20652.96   0.84085474     17366.14     89540.89
FRI 04 MAR 2022  0.4944444     20199.04   0.82477877     16659.74    106200.63
MON 05 SEP 2022  0.5027778     20539.48   0.80762580     16588.21    122788.85
MON 06 MAR 2023  0.5027778     20539.48   0.79020647     16230.43  

In [14]:
swaption.printSwapFloatLeg()

START DATE: MON 04 MAR 2019
MATURITY DATE: MON 04 MAR 2024
SPREAD COUPON (%): 0.0
FLOAT LEG FREQUENCY: TuringFrequencyTypes.QUARTERLY
FLOAT LEG DAY COUNT: TuringDayCountTypes.ACT_360
VALUATION DATE TUE 04 MAR 2014
Floating Flows not calculated.


Copyright (c) 2020 Dominic O'Kane